In [1]:
#IMPORT NECESSARY PACKAGES
import pandas as pd
from datetime import datetime
import CO2_functions
from CO2_functions import *

In [116]:
import importlib
importlib.reload(CO2_functions)
from CO2_functions import *

In [3]:
date1,date2 = get_date_range()
data = get_sql_data("Aug2019_LI_8100_Vent",\
              "Aug2019_Multiplexer","Aug2019_Vent_Anem_Temp",\
              "Aug2019_Picarro",date1,date2,'all','split')
data = drift_correct(data)
for key in data:
    data[key].reset_index(drop=True,inplace=True)

Enter Start Date YYYY-mm-DD:  2019-11-18
Enter End Date YYYY-mm-DD:  2019-11-22


Retrieving LI_vent data
Retrieving Multiplexer data
Retrieving Vent_Anem_Temp data
Retrieving Picarro data
Initializing Drift Correct
Correcting data for LI_Vent
Correcting data for Multiplexer_CO2_1
Correcting data for Multiplexer_CO2_2
Correcting data for Multiplexer_CO2_3
Correcting data for Multiplexer_Weather
Correcting data for Vent_Anem_Temp
Correcting data for Picarro_CO2
Correcting data for Picarro_ANEM


In [123]:
d['Picarro_ANEM']

,Local_DT,EPOCH_TIME,ANEM_Y,ANEM_X,ANEM_Z,Loc,Corrected_ET,Corrected_DT
0,2019-11-18 00:00:00,1.574060e+09,-0.469836,-0.739894,0.193611,NaN,1.574060e+09,2019-11-17 23:57:20.547577
1,2019-11-18 00:00:00,1.574060e+09,-0.520106,-0.707979,0.298085,NaN,1.574060e+09,2019-11-17 23:57:20.647576
2,2019-11-18 00:00:00,1.574060e+09,-0.621927,-0.706881,0.302477,NaN,1.574060e+09,2019-11-17 23:57:20.747575
3,2019-11-18 00:00:00,1.574060e+09,-0.685106,-0.646809,0.282979,NaN,1.574060e+09,2019-11-17 23:57:20.847574
4,2019-11-18 00:00:00,1.574060e+09,-0.693394,-0.679450,0.339450,NaN,1.574060e+09,2019-11-17 23:57:20.947573
...,...,...,...,...,...,...,...,...
4295320,2019-11-22 23:59:59,1.574492e+09,-0.364096,-0.882157,-0.108562,NaN,1.574492e+09,2019-11-22 23:57:15.415788
4295321,2019-11-22 23:59:59,1.574492e+09,-0.275277,-0.840672,-0.116355,NaN,1.574492e+09,2019-11-22 23:57:15.515787
4295322,2019-11-22 23:59:59,1.574492e+09,-0.102728,-0.838730,-0.076549,NaN,1.574492e+09,2019-11-22 23:57:15.615786
4295323,2019-11-22 23:59:59,1.574492e+09,-0.042871,-0.836185,-0.044300,NaN,1.574492e+09,2019-11-22 23:57:15.715785


In [ ]:
plot_refinement_all([[d['Vent_Anem_Temp'],'Corrected_DT','Velocity']])

In [ ]:
df1=df
df1['Velocity'] = df.apply(lambda row: 0 if (row['Velocity'] < 8) & ((row['Corrected_DT'].hour <= 8)|(row['Corrected_DT'].hour >= 19)) else row['Velocity'],axis=1)

In [138]:
df['Corrected_DT'].iloc[0].hour

10